In [2]:
import numpy as np
from scipy.stats import truncnorm
import gym
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import collections
import matplotlib.pyplot as plt

# 1_Cross Entropy Method, CEM

In [3]:
class CEM:
    def __init__(self,
                 n_sequence,
                 elite_ratio,
                 fake_env,
                 upper_bound,
                 lower_bound):
        self.n_sequence = n_sequence
        self.elite_ratio = elite_ratio
        self.fake_env = fake_env
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
    
    def optimize(self, state, init_mean, init_var):
        mean, var = init_mean, init_var
        X = truncnorm(-2, 2, loc=np.zeros_like(mean), scale=np.ones_like(var))
        state = np.tile(state, (self.n_sequence, 1))

        for _ in range(5):
            lb_dist, ub_dist = mean - self.lower_bound, self.upper_bound - mean
            constrained_var = np.minimum(np.minimum(np.square(lb_dist / 2), np.square(ub_dist / 2)), var)
            # Generate action sequences
            action_sequences = [X.rvs() for _ in range(self.n_sequence)] * np.sqrt(constrained_var) + mean
            # Calculate accumulated rewards of each action sequence
            returns = self.fake_env.propagate(state, action_sequences)[:, 0]
            # Select elite sequences
            elites = action_sequences[np.argsort(returns)][-int(self.elite_ratio * self.n_sequence):]

            new_mean = np.mean(elites, axis=0)
            new_var = np.var(elites, axis=0)
            mean = 0.1 * mean + 0.9 * new_mean
            var = 0.1 * var + 0.9 * new_var

        return mean

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2_PETS Algorithm